In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from collections import defaultdict

# Data Sources

https://hifld-geoplatform.opendata.arcgis.com/datasets/6ac5e325468c4cb9b905f1728d6fbf0f_0/data?geometry=70.851%2C-16.829%2C-101.766%2C72.120

https://www.census.gov/data/datasets/2010/demo/popest/modified-race-data-2010.html

https://www.cdc.gov/nchs/nvss/usaleep/usaleep.html

https://apps.bea.gov/iTable/iTable.cfm?reqid=70&step=1&acrdn=6


In [2]:
# Labels
state_dict = {'01':['Alabama','AL','Southeast'],'02':['Alaska','AK','West'],'04':['Arizona','AZ','Southwest'],
              '05':['Arkansas','AR','Southeast'],'06':['California','CA','West'],'08':['Colorado','CO','West'],
              '09':['Connecticut','CT','Northeast'],'10':['Delaware','DE','Northeast'],
              '11':['District of Columbia','DC','Northeast'],'12':['Florida','FL','Southeast'],
              '13':['Georgia','GA','Southeast'],'15':['Hawaii','HI','West'],'16':['Idaho','ID','West'],
              '17':['Illinois','IL','Midwest'],'18':['Indiana','IN','Midwest'],'19':['Iowa','IA','Midwest'],
              '20':['Kansas','KS','Midwest'],'21':['Kentucky','KY','Southeast'],
              '22':['Louisiana','LA','Southeast'],'23':['Maine','ME','Northeast'],
              '24':['Maryland','MD','Northeast'],'25':['Massachusetts','MA','Northeast'],
              '26':['Michigan','MI','Midwest'],'27':['Minnesota','MN','Midwest'],
              '28':['Mississippi','MS','Southeast'],'29':['Missouri','MO','Midwest'],'30':['Montana','MT','West'],
              '31':['Nebraska','NE','Midwest'],'32':['Nevada','NV','West'],
              '33':['New Hampshire','NH','Northeast'],'34':['New Jersey','NJ','Northeast'],
              '35':['New Mexico','NM','Southwest'],'36':['New York','NY','Northeast'],
              '37':['North Carolina','NC','Southeast'],'38':['North Dakota','ND','Midwest'],
              '39':['Ohio','OH','Midwest'],'40':['Oklahoma','OK','Southwest'],'41':['Oregon','OR','West'],
              '42':['Pennsylvania','PA','Northeast'],'44':['Rhode Island','RI','Northeast'],
              '45':['South Carolina','SC','Southeast'],'46':['South Dakota','SD','Midwest'],
              '47':['Tennessee','TN','Southeast'],'48':['Texas','TX','Southwest'],'49':['Utah','UT','West'],
              '50':['Vermont','VT','Northeast'],'51':['Virginia','VA','Southeast'],
              '53':['Washington','WA','West'],'54':['West Virginia','WV','Southeast'],
              '55':['Wisconsin','WI','Midwest'],'56':['Wyoming','WY','West']}
state_dict_r = {state:abr for state,abr,_ in state_dict.values()}
races = ['Total Population','Multiracial','Native Hawaiian/Pacific Islander',
         'American/Alaskan Native','Asian','Black','White','Hispanic']
states = [s[0] for s in state_dict.values()]
regions = ['West','Southwest','Midwest','Southeast','Northeast']
alljurs = ['US'] + regions + states           
stabreviation = [v[1] for v in state_dict.values()]
citerator = iter(('green','blue','purple','red','orange'))
region_color_dict = {r:next(citerator) for r in regions}

In [3]:
# Create dataframe of health care facilities
hospitals = gpd.read_file('Hospitals.shp').drop(['OBJECTID','ID','ADDRESS','CITY','ZIP','ZIP4','TELEPHONE',
                                                 'COUNTRY','WEBSITE','STATE_ID','ALT_NAME','ST_FIPS','LATITUDE',
                                                 'LONGITUDE','NAICS_CODE','SOURCE','VAL_METHOD','VAL_DATE',
                                                 'NAICS_DESC','SOURCEDATE','POPULATION','TTL_STAFF'],axis=1)
hospitals.columns = hospitals.columns.str.lower()
hospitals = hospitals.loc[hospitals.countyfips!='NOT AVAILABLE']
hospitals = hospitals.replace({'GENERAL ACUTE CARE':'Acute Care'})
hospitals['type'] = hospitals['type'].map(lambda h: h.title())

hospital_types = list(pd.Series(hospitals['type'].unique()).str.title()) + ['All Facilities']

citerator = iter(('green','blue','purple','red','orange','cyan','yellow','brown','pink','olive'))
hospital_cmap = {h:next(citerator) for h in hospital_types[:-1]}
hospitals['colors'] = hospitals['type'].map(hospital_cmap)
hospitals['hovers'] = hospitals.name.str.title()+', '+hospitals['type']

hospitals.to_pickle('hospitals.pkl')

In [4]:
# Create dataframe of county demographic data

def long_data(df):
    df_out = df.groupby(['countyfips','race']).sum().unstack()
    df_out.columns = df_out.columns.droplevel()
    df_out = df_out[['Multiracial','Native Hawaiian/Pacific Islander','American/Alaskan Native','Asian',
                     'Black','White']]
    population = df_out.sum(axis=1)
    df_out['Hispanic'] = df.groupby(['countyfips','hispanic']).sum().unstack()['total population'].Hispanic
    df_out = df_out.join(df_out.div(population, axis=0).add_prefix('Percentage '))
    df_out['total population'] = population
    return df_out.fillna(0)

dem_keys = defaultdict(lambda:'Multiracial')
dem_keys.update({1:'White',2:'Black',3:'American/Alaskan Native',4:'Asian',5:'Native Hawaiian/Pacific Islander'})

census = pd.read_csv('stco-mr2010_al_mo.csv')
census = census.append(pd.read_csv('stco-mr2010_mt_wy.csv'))
census['countyfips'] = census.STATE.map(lambda s: f'{s:02}') + census.COUNTY.map(lambda c: f'{c:03}')
census = census.drop(['SEX','AGEGRP','SUMLEV','COUNTY','STATE'],axis=1)
census.columns = ['state','county','hispanic','race','total population','countyfips']

census.race = census.race.map(dem_keys)
census.hispanic = census.hispanic.map({1:'Non-Hispanic',2:'Hispanic'})

# NaN are counties with 0 population for given race
census_long = long_data(census)

# Oglala Lakota missing values, obtained from:
# https://www.census.gov/quickfacts/fact/table/oglalalakotacountysouthdakota/fips#fips
Oglala_Lakota_pop = [13586]
Oglala_Lakota_pct = [0.017,0.001,0.924,0.001,0.004,0.054,0.041]
Oglala_Lakota_num = list((pd.Series(Oglala_Lakota_pct) * Oglala_Lakota_pop[0]).values)
Oglala_Lakota = Oglala_Lakota_num + Oglala_Lakota_pct + Oglala_Lakota_pop

census_long.loc['46102'] = Oglala_Lakota

Kusilvak_pop = [7028]
Kusilvak_pct = [0.025,0.0003 ,0.925, 0.001,0.0006,0.047,0.33]
Kusilvak_num = list((pd.Series(Oglala_Lakota_pct) * Oglala_Lakota_pop[0]).values)
Kusilvak = Kusilvak_num + Kusilvak_pct + Kusilvak_pop
census_long.loc['02158'] = Kusilvak

In [5]:
# create geo dataframe of US counties

cfips = [f'{c:02}' for c in range(1,57)]
counties = gpd.read_file('cb_2018_us_county_500k.shp').drop(['COUNTYFP','COUNTYNS','AFFGEOID',
                                                             'LSAD','ALAND','AWATER'],axis=1)
counties.columns = ['statefps','countyfips','county','geometry']
counties = counties.to_crs(epsg=3857)
counties.set_index('countyfips',inplace=True)

counties = counties.loc[counties.statefps.isin(cfips)]

In [6]:
# Create dataframe of income

income = pd.read_csv('income.csv')[['GeoFips','LineCode','2019']]
income = income[income.LineCode==3].drop('LineCode',axis=1)
income.columns = ['countyfips','Average Income']
income.countyfips = income.countyfips.map(lambda s: f'{s:05}')
income.set_index('countyfips',inplace=True)

# Maui, Kalawao combined in set
income.loc['15005'] = income.loc['15901'] 
income.loc['15009'] = income.loc['15901']

# VA counties & independent cities combined
income.loc['51015'] = income.loc['51907'] # Augusta, Staunton, Waynesboro
income.loc['51790'] = income.loc['51907']
income.loc['51820'] = income.loc['51907'] 

income.loc['51053'] = income.loc['51918'] # Dinwiddie, Colonial Heights, Petersburg
income.loc['51570'] = income.loc['51918'] 
income.loc['51730'] = income.loc['51918'] 

income.loc['51069'] = income.loc['51921'] # Frederick, Winchester
income.loc['51840'] = income.loc['51921'] 
 
income.loc['51149'] = income.loc['51941'] # Prince George, Hopewell
income.loc['51670'] = income.loc['51941'] 

income.loc['51161'] = income.loc['51944'] # Roanoke, Salem 
income.loc['51775'] = income.loc['51944'] 

income.loc['51177'] = income.loc['51951'] # Spotsylvania, Fredericksburg 
income.loc['51630'] = income.loc['51951']

income.loc['51199'] = income.loc['51958'] # York, Poquoson 
income.loc['51735'] = income.loc['51958']

income.loc['51095'] = income.loc['51931'] # James City, Williamsburg 
income.loc['51830'] = income.loc['51931']

income.loc['51165'] = income.loc['51947'] # Rockingham, Harrisonburg 
income.loc['51660'] = income.loc['51947']

income.loc['51175'] = income.loc['51949'] # Southampton, Franklin 
income.loc['51620'] = income.loc['51949']

income.loc['51005'] = income.loc['51903'] # Alleghany, Covington 
income.loc['51580'] = income.loc['51903']

income.loc['51089'] = income.loc['51929'] # Henry, Martinsville 
income.loc['51690'] = income.loc['51929']

income.loc['51081'] = income.loc['51923'] # Greensville, Emporia 
income.loc['51595'] = income.loc['51923']

income.loc['51195'] = income.loc['51955'] # Wise, Norton 
income.loc['51720'] = income.loc['51955']

income.loc['51003'] = income.loc['51901'] # Albemarle, Charlottesville 
income.loc['51540'] = income.loc['51901']

income.loc['51031'] = income.loc['51911'] # Campbell, Lynchburg 
income.loc['51680'] = income.loc['51911']

income.loc['51059'] = income.loc['51919'] # Fairfax, Fairfax City, Falls Church 
income.loc['51600'] = income.loc['51919']
income.loc['51610'] = income.loc['51919']

income.loc['51121'] = income.loc['51933'] # Montgomery, Radford 
income.loc['51750'] = income.loc['51933']

income.loc['51143'] = income.loc['51939'] # Pittsylvania, Danville 
income.loc['51590'] = income.loc['51939']

income.loc['51191'] = income.loc['51953'] # Washington, Bristol 
income.loc['51520'] = income.loc['51953']

income.loc['51153'] = income.loc['51942'] # Prince William, Manassas, Manassas Park 
income.loc['51683'] = income.loc['51942']
income.loc['51685'] = income.loc['51942']

income.loc['51035'] = income.loc['51913'] # Carroll, Galax 
income.loc['51640'] = income.loc['51913']

income.loc['51163'] = income.loc['51945'] # Rockbridge, Buena Vista, Lexington
income.loc['51530'] = income.loc['51945']
income.loc['51678'] = income.loc['51945']

In [7]:
# Create dataframe of life expectancy

expectancy = pd.read_csv('US_B.CSV')[['Age Group','STATE2KX','CNTY2KX','e(x)']]
expectancy = expectancy[expectancy['Age Group']=='Under 1']
expectancy['countyfips'] = expectancy.STATE2KX.map(lambda s: f'{s:02}')+expectancy.CNTY2KX.map(lambda c:f'{c:03}')
expectancy.drop(['Age Group','STATE2KX','CNTY2KX'],axis=1,inplace=True)
expectancy.columns = ['Average Life Expectancy','countyfips']
expectancy = expectancy.groupby('countyfips').mean()
avg_le = expectancy.mean().values[0]

In [59]:
# Merge county demographic and geographic dataframes

H = counties.join(census_long)

In [60]:
# Add number of medical facilities to dataframe

o = hospitals.status=='OPEN'
rename = {'All Facilities Facilities per 100k':'All Facilities per 100k',
          'All Facilities Facilities per Square km':'All Facilities per Square km',
          'Total Population Population per Square km':'Total Population per Square km',
          'Geometry':'geometry'}


facilities = hospitals[o].groupby(['countyfips'])['type'].value_counts().unstack().fillna(0)
facilities['All Facilities'] = facilities.sum(axis=1)

H = H.join(facilities).fillna(0)
H.columns =  H.columns.str.title()
H = H.join((H[hospital_types].div(H['Total Population'],axis=0)*100000).add_suffix(' Facilities per 100k'))
H['Area'] = H.Geometry.area/10**6
H = H.join((H[hospital_types].div(H['Area'],axis=0)).add_suffix(' Facilities per Square km'))

H = H.join((H[races].div(H['Area'],axis=0)).add_suffix(' Population per Square km'))

H['State'] = H.Statefps.map(lambda x: state_dict[x][0])
H['State_Abr'] = H.Statefps.map(lambda x: state_dict[x][1])
H['Region'] = H.Statefps.map(lambda x: state_dict[x][2])
H = H.join(income)
H['Average Income'] = H['Average Income'].astype(int)
H = H.join(expectancy).drop(['Statefps','State_Abr'],axis=1)
H['Average Life Expectancy'].fillna(avg_le,inplace=True)

H['hovers'] = H.County+H.State.map(lambda x: f', {x}')
H['colors'] = H.Region.map(region_color_dict)
H['sizes' ] = np.sqrt(H['Total Population'])/100

H.rename(columns=rename,inplace=True)
H.loc['56039','Average Income'] = 70271 
H.rename(index={'46102':'46113'},inplace=True) # Oglala Lakota has new FIPS

In [61]:
H.to_pickle('H.pkl')

In [62]:
# Create Dataframe of state statistics
H['GDP'] = H['Total Population']*H['Average Income']
H['EDP'] = H['Total Population']*H['Average Life Expectancy']

S = H[races+['State']].groupby('State').sum()
S = S.join(S[races[1:]].div(S['Total Population'], axis=0).add_prefix('Percentage '))
S = S.join(H[['State','Area']].groupby('State').sum())
S = S.join(S[races].div(S['Area'], axis=0).add_suffix(' Population per Square km'))
S = S.join(H[hospital_types+['State']].groupby('State').sum())
S = S.join((S[hospital_types].div(S['Total Population'],axis=0)*100000).add_suffix(' Facilities per 100k'))
S = S.join(S[hospital_types].div(S['Area'],axis=0).add_suffix(' Facilities per Square km'))
S['Average Income'] = H[['State','GDP']].groupby('State').sum().GDP/S['Total Population']
S['Average Life Expectancy'] = H[['State','EDP']].groupby('State').sum().EDP/S['Total Population']
S['Region'] = S.index.map({s:r for s,r in H.set_index(['State','Region']).index.unique()})

S['hovers'] = S.index
S['colors'] = S.Region.map(region_color_dict)
S['sizes' ] = np.sqrt(S['Total Population'])/100

S.rename(columns=rename,inplace=True)
S['State'] = S.index
S.index = S.index.map(state_dict_r)

In [63]:
S.to_pickle('S.pkl')